In [4]:
import os
from io import StringIO

import sys

sys.path.append(r'D:\seagu_OneDrive\Documents\GitHub\PVT-thermal-performance-model')
sys.path.append(r'D:\seagu_OneDrive\Documents\GitHub\PVT-thermal-performance-model\CFD')
sys.path.append(r'D:\seagu_OneDrive\Documents\GitHub\parallel-flow-distribution-pressure-loss')

import numpy as np
import pandas as pd
import openpyxl

import matplotlib.pyplot as plt
import plotly.graph_objects as go

import jou_gen as jg
import bridge_module as bm

import ansys.fluent.core as pyfluent

from CoolProp.CoolProp import PropsSI

import hx_hydraulic as hxhy

In [5]:
main_fp = r'D:\ANSYS Fluent Projects\Hydraulics\V4.5\V4.5_1channel'
Mesh_fp = os.path.join(main_fp, 'mesh')
SR_fp = os.path.join(main_fp, 'SimulationResults')

tui, solver = bm.init_solver(main_fp, '3000')

In [6]:
water_dict = solver.setup.materials.fluid["water-liquid"].get_state()

T_op = 25 + 273.15
p_op = 101325

water_dict['density']['option'] = 'constant'
water_dict['density']['constant'] = PropsSI('D', 'T', T_op, 'P', p_op, 'INCOMP::MPG[0.4]')
water_dict['viscosity']['option'] = 'constant'
water_dict['viscosity']['constant'] = PropsSI('viscosity', 'T', T_op, 'P', p_op, 'INCOMP::MPG[0.4]')

solver.setup.materials.fluid["water-glycol"].set_state(water_dict)

In [7]:
proj_x = 1.858884e-07
proj_y = -0.0192837
proj_z = -0.0160013

coeff_x = proj_x/0.02505798417538183
coeff_y = proj_y/0.02505798417538183
coeff_z = proj_z/0.02505798417538183

In [8]:
# Vdot_list = [0.01, 0.05, 0.1, 0.5, 1., 1.5, 2., 3., 5.] 
Vdot_list = [1., 1.5, 2., 3., 5.] 

In [9]:
nb_it = 500

In [10]:
param_fp = bm.check_folder(os.path.join(SR_fp, 'parametric_laminar_try0'))

In [ ]:
for Vdot in Vdot_list:

    folder_path_case = bm.check_folder(os.path.join(SR_fp, 'parametric_laminar_try0', f'case_{str(Vdot)}Lh'))

    report_names = ['report_outlet_sp', 'report_inlet_sp', 'report_outlet_mdot']
    for report_name in report_names:
        jg.change_report_file_path(tui, report_name, os.path.join(folder_path_case, f"{report_name}.out"))

    Vdot_m3s = Vdot * 1e-3 / 3600 # m3/s
    Ac = 0.0000055427 # m2
    V = Vdot_m3s / Ac # m/s

    jg.change_named_expression(tui, 'V_inlet', V, 'm/s')

    channel = hxhy.duct('rectangular', h = 0.002, w = 0.0028, L = 1.603) # kPa
    PL_guess = 1000 * channel.regular_PL(Vdot, fluid='MPG', glycol_rate=0.4, p=p_op, T=T_op) # Pa

    jg.standard_initialization(tui, 'inlet', PL_guess, coeff_x*V, coeff_y*V, coeff_z*V)

    solver.solution.run_calculation.iterate(number_of_iterations=nb_it)

    jg.write_report(tui, 'mdot', folder_path_case, 'mdot_report')
    jg.write_report(tui, 'sp', folder_path_case, 'sp_report')
    jg.write_residuals_file(tui, folder_path_case, 'residuals')
    jg.write_data(tui, folder_path_case, f'case_{str(Vdot)}Lh')

In [ ]:
folder_path_case

In [13]:
# jg.write_report(tui, 'sp', folder_path_case, 'sp_report')
# jg.write_residuals_file(tui, folder_path_case, 'residuals')
# jg.write_data(tui, folder_path_case, f'case_{str(Vdot)}Lh')

In [45]:
# channel.set_fluid('INCOMP::MPG[0.4]', p_op, T_op)
# channel.set_Vdot(16.)
# channel.compute_Reynolds()
# channel.Re